# Exercises for Lecture 11 (Introduction to TensorFlow)

In [1]:
import datetime
now = datetime.datetime.now()
print("Last executed: " + now.strftime("%Y-%m-%d %H:%M:%S"))

Last executed: 2021-02-04 02:25:43


In [2]:
import numpy as np
import tensorflow as tf

## Exercise 1: Linear regression

### Load housing data

Load the California housing regression dataset.

In [3]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
(m, n)

(20640, 8)

In [4]:
housing.target.shape

(20640,)

In [5]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

The dataset contains 8 features (displayed above) contained in `housing.data` that can be used to predict house prices (in units of $100,000), which are contained in `housing.targets`.

### Set up data (add bias and reshape data)

In [6]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
housing_data_target = housing.target.reshape(-1, 1)

In [7]:
housing_data_target.shape, housing_data_plus_bias.shape

((20640, 1), (20640, 9))

Fit a linear regression model to the data, using a Scikit Learn LinearRegression model and then by computing the analytic solution using numpy and TensorFlow.  

Compare the fitted model between the different computations to ensure you get similar model parameters.

### Exercise 1a: Fit a linear regression model using Scikit Learn

Use the Scikit Learn LinearRegression model.

In [8]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing_data_target)
print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### Exercise 1b: Fit a linear regression model using numpy

Solve the normal equations analytically.

In [9]:
X = housing_data_plus_bias
y = housing_data_target
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


### Exercise 1c: Fit a linear regression model using TensorFlow

Solve the normal equations analytically.

In [10]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_data_target, dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta_tf = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)
print(theta_tf)

tf.Tensor(
[[-3.67372932e+01]
 [ 4.37366009e-01]
 [ 9.47520509e-03]
 [-1.08159676e-01]
 [ 6.48537397e-01]
 [-3.84734449e-06]
 [-3.79239232e-03]
 [-4.19136107e-01]
 [-4.32144403e-01]], shape=(9, 1), dtype=float32)


## Exercise 2: Compute gradients

Compute the partial derivatives of the following function (`my_func`) at `(a,b) = (0.2,0.3)` by:
1. numerical integration 
2. Autodiff in TensorFlow 

Check you get the same answer in both cases.

In [11]:
def my_func(a, b):
    z = 0.0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

### Compute by numerical integration

In [12]:
delta = 0.01
f = my_func
df_da = (f(0.2+delta,0.3) - f(0.2-delta,0.3))/(2*delta)
df_db = (f(0.2,0.3+delta) - f(0.2,0.3-delta))/(2*delta)
df_da, df_db

(-1.1383901861704486, 0.19675140591134815)

### Compute using Autodiff in TensorFlow

In [13]:
a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")

In [14]:
with tf.GradientTape() as t:
    z = 0.0
    for i in range(100):
        z = a * tf.cos(z + i) + z * tf.sin(b - i)
    
gradients = t.gradient(z, [a, b])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.1388494>, <tf.Tensor: shape=(), dtype=float32, numpy=0.19671395>]
